# Discovery notebook for cycling crash data

Crash data from 2010 to 2017

This data came from: https://zenodo.org/records/5603036

Source: https://zenodo.org/records/5603036/files/louisville.zip

Shape: 1273 rows x 54 columns

# Building the data dictionaries


## cycling_safety data
| column name | type | description | value notes | cleaning notes |
|-------------|------|-------------|-------------|----------------|
|  Unnamed: 0 | number | index number for row | | drop | 
| MASTER FILE NUMBER | number | case number for authorities? | | possibly ignore | 
| INVESTIGATING AGENCY | string | agency responding to crash | usually LMPD, but others too | | 
| LOCAL CODE | number | case number for local authority | a couple repeat values | possibly ignore |
| COLLISION STATUS CODE | string | code for case status | usually AC |  | 
| COUNTY NAME | number | county crash occurred in | always "56" (Jefferson County, KY) | drop | 
| ROADWAY NUMBER | alphanumeric | code for state/county/etc roads like KY303 | a lot of null values |  | 
| BLOCK/HOUSE # | number | street number for a location | NAN's and blank values present | | 
| ROADWAY NAME | string | name of primary road where the crash occurred | some blank and null values; there might be some weird long strings | | 
| ROADWAY SUFFIX | string | RD, AVE, LN, WAY, stuff like that | some blank and null values | | 
| ROADWAY DIRECTION CODE | string | S, N, E, W, stuff like that | | | 
| GPS LATITUDE DECIMAL | float/decimal | latitude coordinate | same as "Latitude"? | redundant; drop | 
| GPS LONGITUDE DECIMAL | float/decimal | longitudue coordinate | same as "Longitude"? | redundant; drop | 
| MILEPOINT DERIVED | float | mile marker along road | some null values | maybe useful for hiways? | 
| COLLISION DATE | string | date of collision | 2010-2017 | |
| COLLISION TIME |string | time of collision | integer representing a time. For example 6:30 is 630. But also 10:38 could be 103800. | Drop; redundant and annoying to parse. Get time data from Date column | 
| INTERSECTION ROADWAY # | number | route number for intersecting road|  blanks and NaN's| | 
| INTERSECTION ROADWAY NAME | string | name of intersecting road | blanks and NaN's; at least one problematic string | | 
| INTERSECTION ROADWAY SFX | string | suffix for intersecting road | blanks and NaN's | | 
| BETWEEN STREET ROADWAY # 1 | number | route number for between road #1 | blanks and NaN's | | 
| BETWEEN STREET ROADWAY NAME 1 | string | name for between road #1 | blanks and NaN's | | 
| BETWEEN STREET ROADWAY SFX 1 | string | suffix for between street #1 | blanks and NaN's | | 
| BETWEEN STREET ROADWAY # 2 | number | route number for between road # 2 | blanks and NaN's | | 
| BETWEEN STREET ROADWAY NAME 2 | string | name for between road #2 | blanks and NaN's | | 
| BETWEEN STREET ROADWAY SFX 2 | string | suffix for between road #2 | blanks and NaN's | | 
| UNITS INVOLVED | number | number of units involved in crash (includes bikes, pedestrians, cars, etc.) | | | 
| MOTOR VEHICLES INVOLVED | number | number of motor vehicles involved in crash  | | | 
| KILLED | number | number of people killed in the crash | | | 
| INJURED | number | number of people injured in the crash | | |
| WEATHER CODE | number | numeric code for WEATHER condition | maps 1:1 with WEATHER | redundant? |
| WEATHER | string | human readable for WEATHER CODE | maps 1:1 with WEATHER CODE | | 
| ROADWAY CONDITION CODE | number | numeric code for ROADWAY CONDITION | | | 
| ROADWAY CONDITION | string | human readable for ROWADWAY CONDITION CODE | | |
| HIT & RUN INDICATOR | string | was the crash a hit and run?  | Y/N string indicator | change name to avoid "&"; convert to boolean | 
| ROADWAY TYPE CODE | number | numeric code for ROADWAY TYPE | | | 
| ROADWAY TYPE | string | human readable for ROADWAY TYPE CODE | | | 
| DIRECTIONAL ANALYSIS CODE | number | numeric code for DIRECTIONAL ANALYSIS | | | 
| DIRECTIONAL ANALYSIS | string | human readable for DIRECTIONAL ANALYSIS CODE  | | | 
| MANNER OF COLLISION CODE | number | numeric code for MANNER OF COLLISION  | | | 
| MANNER OF COLLISION | string | human readable for MANNER OF COLLISION CODE | | |
| ROADWAY CHARACTER CODE | number | numeric code for ROADWAY CHARACTER | | | 
| ROADWAY CHARACTER | string | human readable for ROADWAY CHARACTER CODE | | |
| LIGHT CONDITION CODE | number | numeric code for LIGHT CONDITION | | | 
| LIGHT CONDITION | string | human readable string for LIGHT CONDITION CODE | | |
| RAMP FROM ROADWAY ID | number? | road number for off-ramp | all nulls | ignore | 
| RAMP TO ROADWAY ID | number? | road number for on-ramp | all nulls | ignore | 
| SECONDARY COLLISION INDICATOR | string/NAN | was this collision a result of another collision | Y/N string indicator; some null values | is this always 0/False? convert to boolean; investigate null values | 
| hour | int | hour of collision | 24 hour clock | isn't this a repeat of info in COLLISION TIME? | 
| minute | int | minute of collision | normal minutes | isn't this a repeat of info in COLLISION TIME? |
| Date | datestring | another date field? | do these values match with 'COLLISION DATE'? | figure out what the difference is between this field and COLLISION DATE, if any. If they're redundant, ignore one of them |
| Latitude | float | latitude coordinate of crash site | within boundary range for Jefferson County | keep | 
| Longitude | float | longitude coordinate of crash site | within boundary range for Jefferson Cty | keep |
| geometry | POINT | point location in logitude, latitude form | repeated data elsewhere | drop; can reconstruct from other fields |  
| index_right | number | another index value | always 0 | drop | 

## Exploring cycling_safety data

In [111]:
import pandas as pd
import pyparsing as pyp
import matplotlib

DATA = "../data/raw/cycling_safety_louisville.csv"
df = pd.read_csv(DATA)#,index_col=0)
df

,Unnamed: 0,MASTER FILE NUMBER,INVESTIGATING AGENCY,LOCAL CODE,COLLISION STATUS CODE,COUNTY NAME,ROADWAY NUMBER,BLOCK/HOUSE #,ROADWAY NAME,ROADWAY SUFFIX,...,RAMP FROM ROADWAY ID,RAMP TO ROADWAY ID,SECONDARY COLLISION INDICATOR,hour,minute,Date,Latitude,Longitude,geometry,index_right
0,306,1344607,LOUISVILLE METRO POLICE DEPT,8010012297,AC,56,US0031,NaN,BARDSTOWN,RD,...,NaN,NaN,NaN,16,20,2010-02-20 16:20:00,38.231850,-85.707933,POINT (-85.707933333 38.23185),0
1,1506,70803559,LOUISVILLE METRO POLICE DEPT,8010002922,AC,56,NaN,NaN,ZORN,AVE,...,NaN,NaN,N,13,40,2010-01-13 13:40:00,38.273995,-85.696572,POINT (-85.6965716 38.2739947),0
2,1541,70803445,LOUISVILLE METRO POLICE DEPT,8010003041,AC,56,US0042,NaN,BROWNSBORO,RD,...,NaN,NaN,N,10,0,2010-01-13 10:00:00,38.258551,-85.703576,POINT (-85.70357610000001 38.2585512),0
3,1690,70805078,LOUISVILLE METRO POLICE DEPT,8010003577,AC,56,NaN,NaN,PETERSON,AVE,...,NaN,NaN,N,15,50,2010-01-15 15:50:00,38.250012,-85.697265,POINT (-85.6972652 38.2500121),0
4,2876,70811322,LOUISVILLE METRO POLICE DEPT,8010007938,AC,56,NaN,NaN,GEORGETOWN,PL,...,NaN,NaN,N,6,11,2010-02-02 06:11:00,38.195890,-85.793380,POINT (-85.7933803 38.1958905),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,253098,72129917,LOUISVILLE METRO POLICE DEPT,8017098682,AC,56,,6600,GRADE,LN,...,NaN,NaN,N,7,7,2017-12-05 07:07:00,38.153815,-85.733644,POINT (-85.73364410000001 38.1538153),0
1269,253931,72127942,LOUISVILLE METRO POLICE DEPT,8017101406,AC,56,,,JEANINE,DR,...,NaN,NaN,N,17,9,2017-12-14 17:09:00,38.163618,-85.688008,POINT (-85.6880079 38.1636178),0
1270,254491,72130250,LOUISVILLE METRO POLICE DEPT,8017102897,AC,56,KY0864,,POPLAR LEVEL,RD,...,NaN,NaN,N,10,0,2017-12-19 10:00:00,38.160030,-85.671480,POINT (-85.6714798 38.1600301),0
1271,254702,72131366,LOUISVILLE METRO POLICE DEPT,8017103524,AC,56,,,WOODGATE,LN,...,NaN,NaN,N,19,56,2017-12-21 19:56:00,38.198257,-85.626309,POINT (-85.62630919999999 38.1982569),0


Column names can be organized into groups of related data
### Date / Time columns
['COLLISION DATE', 'COLLISION TIME', 'hour', 'minute', 'Date']
### Geolocation columns
['GPS LATITUDE DECIMAL', 'GPS LONGITUDE DECIMAL', 'Latitude', 'Longitude', 'geometry']
### Address columns
['COUNTY NAME', 'ROADWAY NUMBER', 'BLOCK/HOUSE #', 'ROADWAY NAME', 'ROADWAY SUFFIX',
    'ROADWAY DIRECTION CODE', 'MILEPOINT DERIVED','INTERSECTION ROADWAY #', 'INTERSECTION ROADWAY NAME',
       'INTERSECTION ROADWAY SFX', 'BETWEEN STREET ROADWAY # 1',
       'BETWEEN STREET ROADWAY NAME 1', 'BETWEEN STREET ROADWAY SFX 1',
       'BETWEEN STREET ROADWAY # 2', 'BETWEEN STREET ROADWAY NAME 2',
       'BETWEEN STREET ROADWAY SFX 2', 'RAMP FROM ROADWAY ID', 'RAMP TO ROADWAY ID']
### Code columns
['LOCAL CODE', 'COLLISION STATUS CODE', 'WEATHER CODE', 
 'ROADWAY CONDITION CODE', 'ROADWAY TYPE CODE', 'DIRECTIONAL ANALYSIS CODE',
 'MANNER OF COLLISION CODE', 'ROADWAY CHARACTER CODE', 'LIGHT CONDITION CODE']
    # Some codes are paired with a human_readable string
['WEATHER', 'ROADWAY CONDITION', 'ROADWAY TYPE', 'DIRECTIONAL ANALYSIS', 'MANNER OF COLLISION',
     'ROADWAY CHARACTER', 'LIGHT CONDITION']
   #### Other codes are stand alone:
['LOCAL CODE', 'COLLISION STATUS CODE']

### Other info
['Unnamed: 0', 'MASTER FILE NUMBER', 'INVESTIGATING AGENCY', 'LOCAL CODE', 'COLLISION STATUS CODE',
  'COUNTY NAME','UNITS INVOLVED', 'MOTOR VEHICLES INVOLVED', 'KILLED', 'INJURED', 'HIT & RUN INDICATOR',
  'SECONDARY COLLISION INDICATOR', 'index_right']


# Columns



### Date and time columns
`['COLLISION DATE', 'COLLISION TIME', 'hour', 'minute', 'Date']`

In [112]:
df[['Date', 'COLLISION DATE']].agg(("min", "max"))

,Date,COLLISION DATE
min,2010-01-13 10:00:00,1/10/2011
max,2017-12-22 21:51:00,9/9/2016


In [113]:
df[['COLLISION TIME', 'hour', 'minute']].agg(("min", "max"))

,COLLISION TIME,hour,minute
min,0,0,0
max,100009,23,59


In [114]:
df[['Date', 'COLLISION DATE', 'COLLISION TIME']]
# These seem to match up. It shouldn't be too hard to write code to verify this.

,Date,COLLISION DATE,COLLISION TIME
0,2010-02-20 16:20:00,2/20/2010,1620
1,2010-01-13 13:40:00,1/13/2010,1340
2,2010-01-13 10:00:00,1/13/2010,100008
3,2010-01-15 15:50:00,1/15/2010,1550
4,2010-02-02 06:11:00,2/2/2010,611
...,...,...,...
1268,2017-12-05 07:07:00,12/5/2017,707
1269,2017-12-14 17:09:00,12/14/2017,1709
1270,2017-12-19 10:00:00,12/19/2017,100002
1271,2017-12-21 19:56:00,12/21/2017,1956


In [115]:
# Bringing in pyparsing to break up the values in Date and COLLISION DATE columns
integer = pyp.Word(pyp.nums).set_name("integer")

COLLISION_DATE_expr = integer("month") + '/' + integer("day") + '/' + integer('year')
Date_expr = (integer('year') + '-' + integer('month') + '-' + integer('day') +
             integer('hour') + ':' + integer('minute') + ":" + integer('second'))

def parse(string:str, parsing_expression):
    parsed = parsing_expression.parse_string(string).as_dict()
    return {key:int(value) for key, value in parsed.items()}

parse("12/1/2021", COLLISION_DATE_expr)

{'month': 12, 'day': 1, 'year': 2021}

In [116]:
r = df.iloc[201]
CD = parse(r['COLLISION DATE'], COLLISION_DATE_expr)
Date = parse(r['Date'], Date_expr)
assert Date['month'] == CD['month']
assert Date['year'] == CD['year']
assert Date['day'] == CD['day']
CD, Date

def check_CD_and_Date(df):
    for location in range(len(df)):
        row = df.iloc[location]
        collision_date = parse(row['COLLISION DATE'], COLLISION_DATE_expr)
        date = parse(row['Date'], Date_expr)
        for name in ['month', 'day', 'year']:
            assert date[name] == collision_date[name]
    else:
        return True

check_CD_and_Date(df)
# All of these records match. 
# I will drop COLLISION DATE and rely on information from Date column


True

In [117]:
times = df[['Date', 'COLLISION TIME','hour', 'minute']]
times
# These seem to match up. 
# Checking random slices:
times.iloc[200:230]

,Date,COLLISION TIME,hour,minute
200,2011-04-29 13:42:00,1342,13,42
201,2011-04-08 22:07:00,2207,22,7
202,2011-05-08 13:53:00,1353,13,53
203,2011-05-10 19:01:00,1901,19,1
204,2011-05-12 12:30:00,1230,12,30
205,2011-05-14 13:09:00,1309,13,9
206,2011-05-20 17:53:00,1753,17,53
207,2011-05-20 16:29:00,1629,16,29
208,2011-05-20 20:25:00,2025,20,25
209,2011-05-17 23:13:00,2313,23,13


In [118]:
times[(times.hour <= 0) & times.minute > 0]
# On visual inspection hour/minute/COLLISION TIME columns seem to match up
# These are the values I did not quite understand, but they are few and
# they also match.
# I will drop COLLISION TIME and rely on Date column info for all my time/date data

,Date,COLLISION TIME,hour,minute
165,2011-01-27 00:45:00,45,0,45
222,2011-06-05 00:45:00,45,0,45
304,2011-11-24 00:51:00,51,0,51
379,2012-04-05 00:05:00,5,0,5
474,2012-09-26 00:47:00,47,0,47
495,2012-12-21 00:21:00,21,0,21
499,2013-01-08 00:25:00,25,0,25
637,2013-08-30 00:05:00,5,0,5
754,2014-06-01 00:43:00,43,0,43
938,2015-07-17 00:35:00,35,0,35


In [119]:
def check_Date_and_hour_minute(df):
    for location in range(len(df)):
        row = df.iloc[location]
        date = parse(row['Date'], Date_expr)
        hour = int(row['hour'])
        minute = int(row['minute'])
        assert date['hour'] == hour
        assert date['minute'] == minute
    else:
        return True

check_Date_and_hour_minute(df)
# These all match as well. 
# I will also drop/ignore "hour" and "minute" columns.

True

### Geolocation columns

`['GPS LATITUDE DECIMAL', 'GPS LONGITUDE DECIMAL', 'Latitude', 'Longitude', 'geometry']`


In [120]:

df[['GPS LONGITUDE DECIMAL', 'Longitude', 'GPS LATITUDE DECIMAL', 'Latitude', 'geometry']]
# POINT data is more precise than other lat/long data

,GPS LONGITUDE DECIMAL,Longitude,GPS LATITUDE DECIMAL,Latitude,geometry
0,-85.707933,-85.707933,38.231850,38.231850,POINT (-85.707933333 38.23185)
1,-85.696572,-85.696572,38.273995,38.273995,POINT (-85.6965716 38.2739947)
2,-85.703576,-85.703576,38.258551,38.258551,POINT (-85.70357610000001 38.2585512)
3,-85.697265,-85.697265,38.250012,38.250012,POINT (-85.6972652 38.2500121)
4,-85.793380,-85.793380,38.195890,38.195890,POINT (-85.7933803 38.1958905)
...,...,...,...,...,...
1268,-85.733644,-85.733644,38.153815,38.153815,POINT (-85.73364410000001 38.1538153)
1269,-85.688008,-85.688008,38.163618,38.163618,POINT (-85.6880079 38.1636178)
1270,-85.671480,-85.671480,38.160030,38.160030,POINT (-85.6714798 38.1600301)
1271,-85.626309,-85.626309,38.198257,38.198257,POINT (-85.62630919999999 38.1982569)


In [121]:

df[df['GPS LATITUDE DECIMAL'] != df['Latitude']]
# This returns an empty dataframe. Great. These two fields are identical
# Test the same for longitude
df[df['GPS LONGITUDE DECIMAL'] != df['Longitude']]
# Same result: an empty data frame
# Conclusion: keep the columns with the less cumbersome names: Latitude and Longitude

,Unnamed: 0,MASTER FILE NUMBER,INVESTIGATING AGENCY,LOCAL CODE,COLLISION STATUS CODE,COUNTY NAME,ROADWAY NUMBER,BLOCK/HOUSE #,ROADWAY NAME,ROADWAY SUFFIX,...,RAMP FROM ROADWAY ID,RAMP TO ROADWAY ID,SECONDARY COLLISION INDICATOR,hour,minute,Date,Latitude,Longitude,geometry,index_right


In [122]:
# Bringing in pyparsing to look at 'geometry' column
Float = pyp.Word(pyp.nums + "-.")
point_expr = Float + Float


In [123]:

df['geometry'][1271]
# point_expr.search_string(test).asList()[0]
# ^^^^ Test to see if the parsing works as expected: It does.

'POINT (-85.62630919999999 38.1982569)'

In [124]:
def parse_point(point:str):
    longtitude, latitude = point_expr.search_string(point).asList()[0]
    return {"Longitude": float(longtitude), "Latitude":float(latitude)}
parse_point(test) # test works
    

{'Longitude': -85.6263092, 'Latitude': 38.1982569}

In [125]:
def validate_point(location):
    record = df.iloc[location]
    point = parse_point(record['geometry'])
    return ( point['Latitude'] == record['Latitude'] and 
            point['Longitude'] == record['Longitude'])
                 
all(validate_point(loc) for loc in range(len(df)))
# Great. All the geolocation data matches. I'll drop the geometry column for now, since I can always 
# generate a new one that will be accurate probably

True

### "... CODE" columns

`['LOCAL CODE', 'COLLISION STATUS CODE']`

`['WEATHER CODE', 'ROADWAY CONDITION CODE', 'ROADWAY TYPE CODE', 'DIRECTIONAL ANALYSIS CODE',
 'MANNER OF COLLISION CODE', 'ROADWAY CHARACTER CODE', 'LIGHT CONDITION CODE']`

`['WEATHER', 'ROADWAY CONDITION', 'ROADWAY TYPE', 'DIRECTIONAL ANALYSIS', 'MANNER OF COLLISION',
'ROADWAY CHARACTER', 'LIGHT CONDITION']`

In [175]:
LC = df['LOCAL CODE']
LC.info()
len(LC.unique())
# Some of these are not unique.
vc = LC.value_counts() # get value counts for each LOCAL CODE value
vc[vc.values > 1].keys().to_list() # select the LOCAL CODE values with more than one row


<class 'pandas.core.series.Series'>
RangeIndex: 1273 entries, 0 to 1272
Series name: LOCAL CODE
Non-Null Count  Dtype 
--------------  ----- 
1273 non-null   object
dtypes: object(1)
memory usage: 10.1+ KB


['8013052741', '8017098682']

In [186]:
data = df[['LOCAL CODE', 'COLLISION STATUS CODE']]
data[data['LOCAL CODE'].isin(['8013052741', '8017098682'])]

,LOCAL CODE,COLLISION STATUS CODE
593,8013052741,AC
598,8013052741,AC
1267,8017098682,AC
1268,8017098682,AC


In [181]:
CSC = df['COLLISION STATUS CODE']
CSC.info()
CSC[CSC.values != "AC"]


<class 'pandas.core.series.Series'>
RangeIndex: 1273 entries, 0 to 1272
Series name: COLLISION STATUS CODE
Non-Null Count  Dtype 
--------------  ----- 
1273 non-null   object
dtypes: object(1)
memory usage: 10.1+ KB


14      RE
60      RE
251     RE
350     RE
506     RE
585     RE
644     RE
666     RE
686     RE
781     RE
979     RE
1039    RE
1056    RE
1071    RE
1088    RE
1105    RE
1231    RE
Name: COLLISION STATUS CODE, dtype: object

In [204]:
duplicates = df[df['LOCAL CODE'].isin(['8013052741', '8017098682'])]
dup = duplicates[duplicates["LOCAL CODE"] == '8013052741']#, '8017098682']:
for column in df.columns:
    if dup.iloc[0][column] == dup.iloc[1][column]:
        dup = dup.drop(column, axis=1)
dup
   
    

,Unnamed: 0,MASTER FILE NUMBER,ROADWAY NUMBER,BLOCK/HOUSE #,ROADWAY NAME,ROADWAY SUFFIX,ROADWAY DIRECTION CODE,GPS LATITUDE DECIMAL,GPS LONGITUDE DECIMAL,MILEPOINT DERIVED,...,ROADWAY TYPE CODE,ROADWAY TYPE,MANNER OF COLLISION CODE,MANNER OF COLLISION,RAMP FROM ROADWAY ID,RAMP TO ROADWAY ID,SECONDARY COLLISION INDICATOR,Latitude,Longitude,geometry
593,107635,1187288,NaN,NaN,W BROADWAY,NaN,NaN,38.246800,-85.766750,NaN,...,5.0,LOCAL STREET,1.0,ANGLE,NaN,NaN,NaN,38.246800,-85.766750,POINT (-85.76675 38.2468)
598,107719,71331086,US0150,NaN,BROADWAY,NaN,W,38.246946,-85.766813,1.937,...,2.0,FEDERAL,9.0,SINGLE VEHICLE,NaN,NaN,N,38.246946,-85.766813,POINT (-85.7668132 38.2469465)


In [208]:
dup = duplicates[duplicates["LOCAL CODE"] == '8017098682']
for column in df.columns:
    if dup.iloc[0][column] == dup.iloc[1][column]:
        dup = dup.drop(column, axis=1)
dup

,Unnamed: 0,MASTER FILE NUMBER,INTERSECTION ROADWAY #,INTERSECTION ROADWAY NAME,INTERSECTION ROADWAY SFX,BETWEEN STREET ROADWAY # 1,BETWEEN STREET ROADWAY NAME 1,BETWEEN STREET ROADWAY SFX 1,BETWEEN STREET ROADWAY # 2,BETWEEN STREET ROADWAY NAME 2,BETWEEN STREET ROADWAY SFX 2,RAMP FROM ROADWAY ID,RAMP TO ROADWAY ID
1267,253011,72123368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1268,253098,72129917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### What to do with these duplicate records?
Probably ignore them for now.
I may go back and manually combine them if I can

In [159]:
# X CODE : HUMAN READABLE pairs

human_readable = ['WEATHER', 'ROADWAY CONDITION', 'ROADWAY TYPE', 'DIRECTIONAL ANALYSIS', 'MANNER OF COLLISION',
     'ROADWAY CHARACTER', 'LIGHT CONDITION']
codes = {H:" ".join((H, "CODE")) for H in human_readable}
codes

{'WEATHER': 'WEATHER CODE',
 'ROADWAY CONDITION': 'ROADWAY CONDITION CODE',
 'ROADWAY TYPE': 'ROADWAY TYPE CODE',
 'DIRECTIONAL ANALYSIS': 'DIRECTIONAL ANALYSIS CODE',
 'MANNER OF COLLISION': 'MANNER OF COLLISION CODE',
 'ROADWAY CHARACTER': 'ROADWAY CHARACTER CODE',
 'LIGHT CONDITION': 'LIGHT CONDITION CODE'}

In [202]:
df[codes.values()].apply(lambda x:x.unique())

WEATHER CODE                           [2.0, 9.0, 3.0, 6.0, 97.0, 1.0]
ROADWAY CONDITION CODE                       [1.0, 5.0, 4.0, 6.0, 2.0]
ROADWAY TYPE CODE            [2.0, 5.0, 7.0, 1.0, 4.0, nan, 99.0, 3.0]
DIRECTIONAL ANALYSIS CODE                                  [2.0, 37.0]
MANNER OF COLLISION CODE           [1.0, 9.0, 8.0, 3.0, 5.0, 4.0, 7.0]
ROADWAY CHARACTER CODE                  [4.0, 3.0, 6.0, 1.0, 2.0, 5.0]
LIGHT CONDITION CODE               [2.0, 5.0, 6.0, 3.0, 1.0, 4.0, 7.0]
dtype: object

In [154]:
# Test each X CODE : X pair: They should be a 1:1 mapping, so the count of unique items for each
# human readable should equal the count of unique values for the numeric codes
all((len(df[key].unique()) == len(df[value].unique())) for key, value in codes.items())

True

In [128]:
df['ROADWAY TYPE CODE'].value_counts(dropna=False)
# Has some null values
# NAN roadway type code -> NONE OF THE ABOVE in ROADWAY TYPE

ROADWAY TYPE CODE
5.0     665
7.0     270
2.0     255
1.0      48
99.0     19
4.0      11
NaN       4
3.0       1
Name: count, dtype: int64

In [158]:
roadway = df[['ROADWAY TYPE CODE', 'ROADWAY TYPE']]
roadway[roadway['ROADWAY TYPE CODE'].isnull()]
# 4 records have a null ROADWAY TYPE CODE and null ROADWAY TYPE?

,ROADWAY TYPE CODE,ROADWAY TYPE
125,NaN,NaN
129,NaN,NaN
261,NaN,NaN
373,NaN,NaN


### Address information

`['COUNTY NAME', 'ROADWAY NUMBER', 'BLOCK/HOUSE #', 'ROADWAY NAME', 'ROADWAY SUFFIX',
    'ROADWAY DIRECTION CODE', 'MILEPOINT DERIVED','INTERSECTION ROADWAY #', 'INTERSECTION ROADWAY NAME',
       'INTERSECTION ROADWAY SFX', 'BETWEEN STREET ROADWAY # 1',
       'BETWEEN STREET ROADWAY NAME 1', 'BETWEEN STREET ROADWAY SFX 1',
       'BETWEEN STREET ROADWAY # 2', 'BETWEEN STREET ROADWAY NAME 2',
       'BETWEEN STREET ROADWAY SFX 2', 'RAMP FROM ROADWAY ID', 'RAMP TO ROADWAY ID']`

In [129]:
# COUNTY NAME column
CN = df['COUNTY NAME']
CN.unique()
# Always "56"
# Ignore this? All reports are in Jefferson County, KY


array([56])

In [130]:
df['ROADWAY NUMBER'].value_counts(dropna=False)
# Many of these are NAN or blank
df['ROADWAY NUMBER'].unique()

array(['US0031  ', nan, 'US0042  ', 'KY1020  ', 'US0031E ', 'US0150  ',
       'KY2051  ', 'KY1631  ', 'KY2053  ', 'KY1747  ', 'US0031W ',
       'KY0061  ', 'KY1865  ', 'US0060  ', 'KY1065  ', 'I 0264  ',
       'KY0155  ', 'KY2048  ', 'KY0864  ', 'US0060A ', 'KY1931  ',
       'KY2049  ', 'KY1447  ', 'KY2860  ', 'KY2050  ', 'KY1934  ',
       'KY2052  ', 'KY1703  ', 'KY1450  ', 'KY3064  ', 'KY1819  ',
       'KY2054  ', 'KY1932  ', 'KY1727  ', 'KY0146  ', 'KY0907  ',
       'KY0841  ', 'KY2251  ', 'KY3082  ', 'I 0064  ', 'KY0913  ',
       'KY2845  ', 'I 0065  ', 'KY2055  ', 'KY3077  ', 'KY1230  ',
       'KY1142  ', 'KY0148  ', '        ', 'I 0265  '], dtype=object)

In [131]:
df['BLOCK/HOUSE #'].value_counts(dropna=False)
# Building address numbers
# Some crash reports don't have any value here, or a null value
# My assumption is that sometimes crashes don't occur near any particular address

BLOCK/HOUSE #
NaN       1142
            35
600.0        2
4200         2
900.0        2
          ... 
1500.0       1
8020.0       1
9120.0       1
2216.0       1
131.0        1
Name: count, Length: 90, dtype: int64

In [132]:
# ROADWAY NAME column 
RN = df['ROADWAY NAME']
RN.info()


<class 'pandas.core.series.Series'>
RangeIndex: 1273 entries, 0 to 1272
Series name: ROADWAY NAME
Non-Null Count  Dtype 
--------------  ----- 
1264 non-null   object
dtypes: object(1)
memory usage: 10.1+ KB


In [133]:
# ROADWAY SUFFIX column 
RS = df['ROADWAY SUFFIX']
RS.unique()
RS.value_counts(dropna=False)
# Some blank values and NaN's

ROADWAY SUFFIX
ST       378
RD       227
AVE      202
LN        90
NaN       81
HWY       79
DR        67
BLVD      49
PKWY      33
TRL       11
CT        10
WAY        9
PL         8
LOOP       7
TRCE       4
TPKE       4
           3
CIR        3
BYP        3
ALY        2
PARK       1
TER        1
PLZ        1
Name: count, dtype: int64

In [134]:
# MILEPOINT DERIVED column
import statistics
MP = df['MILEPOINT DERIVED']
# This has some null values
MP.dropna().agg(("min", "max", statistics.mean, statistics.median, statistics.mode, statistics.stdev))

# Probably not useful for me. Likely will drop column.

min         0.000000
max       124.526000
mean        4.037019
median      1.170000
mode        0.000000
stdev       6.485810
Name: MILEPOINT DERIVED, dtype: float64

In [135]:
# INTERSECTION ROADWAY columns
IRW = df[['INTERSECTION ROADWAY #', 'INTERSECTION ROADWAY NAME', 'INTERSECTION ROADWAY SFX']]
IRW
# Intersecting road where crash occurred. Same types of info as ROADWAY NUMBER, ROADWAY NAME, ROADWAY SUFFIX
# One long string at index [1]; might there be others? 
# NaN's and blanks seem significant, because there may be no intersection for some crashes.

,INTERSECTION ROADWAY #,INTERSECTION ROADWAY NAME,INTERSECTION ROADWAY SFX
0,NaN,DEERPARK,NaN
1,I 0071,I71 N EXIT2 OFF RAMP TO ZORN AVE,NaN
2,NaN,JANE,ST
3,NaN,NaN,NaN
4,NaN,CONN,ST
...,...,...,...
1268,NaN,NaN,NaN
1269,,WOODED,WAY
1270,,JEFFERSON,BLVD
1271,,ARJAY,LN


In [136]:
# BETWEEN STREET columns
BWS = df[['BETWEEN STREET ROADWAY # 1', 'BETWEEN STREET ROADWAY NAME 1', 'BETWEEN STREET ROADWAY SFX 1',
       'BETWEEN STREET ROADWAY # 2', 'BETWEEN STREET ROADWAY NAME 2', 'BETWEEN STREET ROADWAY SFX 2']]
BWS[200:210] # Just looking at random slices.
# Similar information as ROADWAY NUMBER, ROADWAY NAME, INTERSECTION ROADWAY #, etc.
# These seem to report crashes that occur between two roads.
# Check to see how this affects the primary/intersecting road information in the other columns

,BETWEEN STREET ROADWAY # 1,BETWEEN STREET ROADWAY NAME 1,BETWEEN STREET ROADWAY SFX 1,BETWEEN STREET ROADWAY # 2,BETWEEN STREET ROADWAY NAME 2,BETWEEN STREET ROADWAY SFX 2
200,NaN,NaN,NaN,NaN,NaN,NaN
201,KY2049,CRUMS,LN,NaN,KRISTIN,WAY
202,NaN,NaN,NaN,NaN,NaN,NaN
203,KY3217,37TH,ST,NaN,36TH,ST
204,NaN,NaN,NaN,NaN,NaN,NaN
205,NaN,NaN,NaN,NaN,NaN,NaN
206,NaN,BILLTOWN,RD,NaN,VANTAGE PLAZA,DR
207,NaN,NaN,NaN,NaN,NaN,NaN
208,NaN,NaN,NaN,NaN,NaN,NaN
209,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# RAMP FROM/TO ROADWAY column
df[['RAMP FROM ROADWAY ID', 'RAMP TO ROADWAY ID']].info()
# All the values for this column are null. Probably can ignore but check if similar column exists in other data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   RAMP FROM ROADWAY ID  0 non-null      float64
 1   RAMP TO ROADWAY ID    0 non-null      float64
dtypes: float64(2)
memory usage: 20.0 KB


### Other columns
`['Unnamed: 0', 'MASTER FILE NUMBER', 'INVESTIGATING AGENCY', 'LOCAL CODE', 'COLLISION STATUS CODE',
  'COUNTY NAME','UNITS INVOLVED', 'MOTOR VEHICLES INVOLVED', 'KILLED', 'INJURED', 'HIT & RUN INDICATOR',
  'SECONDARY COLLISION INDICATOR', 'index_right']`

In [151]:
# Unnamed: 0 column
unnamed_column = df['Unnamed: 0']
len(unnamed_column) == len(unnamed_column.unique())
# This looks to be an index column.
# It was probably used in the original study.
# It is unique for each row, and I don't know what it means. 
# I'll ignore it and make my own fresh index

True

In [139]:
# index_right column
df.index_right
df.index_right.unique()
# This column is all zeros. I will ignore it entirely. 

array([0])

In [152]:
# MASTER FILE NUMBER column
MFN = df['MASTER FILE NUMBER']
len(MFN.unique()), len(MFN), len(df)
# This looks like a unique identifier for each crash report. 
# I will keep this column


(1273, 1273, 1273)

In [141]:
# INVESTIGATING AGENCY
IA = df['INVESTIGATING AGENCY']
IA.unique()
IA.value_counts(dropna=False)

INVESTIGATING AGENCY
LOUISVILLE METRO POLICE DEPT      1160
SHIVELY POLICE DEPARTMENT           40
ST. MATTHEWS POLICE DEPARTMENT      29
JEFFERSONTOWN POLICE DEPT           24
UNIV. OF LOUISVILLE POLICE           7
INDIAN HILLS POLICE DEPARTMENT       5
PROSPECT POLICE DEPARTMENT           2
GRAYMOOR-DEVONDALE POLICE DEPT       2
ANCHORAGE POLICE DEPARTMENT          1
WEST BUECHEL POLICE DEPT.            1
NORTHFIELD POLICE DEPARTMENT         1
AUDUBON PARK POLICE DEPARTMENT       1
Name: count, dtype: int64

In [142]:
# LOCAL CODE column
LC = df['LOCAL CODE']
LC.value_counts(dropna=False)
# Not all of these are unique!




LOCAL CODE
8013052741    2
8017098682    2
8010012297    1
8014072189    1
8014076662    1
             ..
8012053577    1
8012052891    1
8012070216    1
8012067992    1
8017103882    1
Name: count, Length: 1271, dtype: int64

In [143]:
# COLLISION STATUS CODE column
CSC = df['COLLISION STATUS CODE']
CSC.value_counts(dropna=False)
# Figure out that these codes mean


COLLISION STATUS CODE
AC    1256
RE      17
Name: count, dtype: int64

In [144]:
# UNITS INVOLVED column
UI = df["UNITS INVOLVED"]
UI.value_counts(dropna=False)


UNITS INVOLVED
2    1253
3      15
4       3
1       2
Name: count, dtype: int64

In [145]:
# MOTOR VEHICLES INVOLVED column
MVI = df['MOTOR VEHICLES INVOLVED']
MVI.value_counts()


MOTOR VEHICLES INVOLVED
1    1247
2      22
3       4
Name: count, dtype: int64

In [146]:
# KILLED/INJURED/HIT & RUN INDICATOR/SECONDARY COLLISION INDICATOR columns
df["KILLED"].value_counts(dropna=False)

KILLED
0    1262
1      11
Name: count, dtype: int64

In [147]:
df['INJURED'].value_counts(dropna=False)

INJURED
1    863
0    392
2     15
3      2
5      1
Name: count, dtype: int64

In [148]:
df['HIT & RUN INDICATOR'].value_counts(dropna=False)

HIT & RUN INDICATOR
N    1078
Y     195
Name: count, dtype: int64

In [153]:
df['SECONDARY COLLISION INDICATOR'].value_counts(dropna=False)

SECONDARY COLLISION INDICATOR
N      1188
NaN      72
Y        13
Name: count, dtype: int64